Import Area

In [1]:
# cell 1
import csv
import difflib
import pandas as pd
from tqdm import tqdm
import requests, codecs

Read and Load from Sanskrit vritham database

In [2]:
# cell 2
# run cell 1 before this cell
filename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/db/data.csv' # sanskrit only database
#filename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/db/ml_data.csv' # sanskrit + malayalam database
csvresponse = requests.get(filename)
csvfile = codecs.iterdecode(csvresponse.iter_lines(), 'UTF-8')

outputlist = []
keydictionary = {}
 
def check_GL(pattern): # return True if input only contains GL else False
    pattern = pattern.upper()
    for akshara in pattern:
        if akshara not in ('G','L'):return False
    return True
    
def check_maathra(akshara_pattern): # return True if input only contains NJYSBMTRGL else False
    akshara_pattern=akshara_pattern.upper()
    for akshara in akshara_pattern:
        if akshara not in ('N','J','Y','S','B','M','T','R','G','L'):return False
    return True
 
def Compute_maathra(akshara_pattern): # calculate maathra input NJYSBMTRGL string/list
    akshara_pattern=akshara_pattern.upper()
    Maathra_table = {'N':3,'J':4,'Y':5,'S':4,'B':4,
              'M':6,'T':5,'R':5,'G':2,'L':1}
    maathra = 0
    for akshara in akshara_pattern:
        maathra += Maathra_table.get(akshara,0)
    return maathra
 
def get_maathra(text): # get NJYSBMTRGL from GL string
    triplets = {'LLL':'N','LLG':'S','LGL':'J','LGG':'Y',
                'GLL':'B','GLG':'R','GGL':'T','GGG':'M'}
    output = ''
    for i in range(0,len(text),3):
        if len(text[i:i+3]) == 3:output += triplets.get(text[i:i+3].upper(),'')
        else:output += text[i:i+3].upper()
    return output

def calculatemathra(arr): # calculate maathra input GL string/list
    mathra = 0; scores = {'G':2,'L':1}
    for x in arr:
        mathra += scores.get(x.upper(),0)
    return mathra

def getGLtriplet(character):
    valid = ['N','S','J','Y','B','R','T','M']
    if character.upper() not in valid:return character.upper()
    else:return str('{0:03b}'.format(valid.index(character.upper()))).replace('0','L').replace('1','G')

def converttoGL(string):
    output = ''
    for character in string:
        output+=getGLtriplet(character)
    return output
 
csvreader = csv.reader(csvfile)
for row in csvreader:
    if len(row[-1])>0:
        keydictionary[row[-1].strip().upper()] = row[:-1]
        keydictionary[row[-1].strip().upper()].append(Compute_maathra(row[-1].strip().upper()))
        tlist = list(row)
        tlist.append(Compute_maathra(row[-1].strip().upper()))
        outputlist.append(tlist)
 
with open('out.csv', 'w', encoding='utf-8') as outputfile:
    csvwriter = csv.writer(outputfile)
    csvwriter.writerows(outputlist)

some statistical calculations and testing

In [ ]:
# cell 3
# run cell 1 and 2 before this cell
# matrix maker 2mer
# find count of occurance of kmer (2mer) NN, BN, GL etc...
def create_matrix(): # 2-mer array only :D
    matrix = []
    entries = ['S','N','J','B','R','Y','T','M','G','L']
    for entry in entries:
        matrix.append([0,0,0,0,0,0,0,0,0,0])
    return matrix

def create_kmers(data,merlength=2):
    mers = []
    for text in data:
        for index in range(len(text)):
            current_mer = text[index:index+merlength]
            if len(current_mer)==merlength:mers.append(current_mer)
    return mers
entries = ['S','N','J','B','R','Y','T','M','G','L']
vritham_matrix = create_matrix()
kmers = create_kmers(keydictionary.keys())
for kmer in kmers:
    x = entries.index(kmer[0].upper())
    y = entries.index(kmer[1].upper())
    vritham_matrix[y][x]+=1
y_arr = [' '];y_arr.extend(entries)
y = [print(str(yval),end="\t")for yval in y_arr];print()
for row_ind in range(len(vritham_matrix)):
    x_d = [entries[row_ind]];x_d.extend(vritham_matrix[row_ind])
    x = [print(str(xval),end="\t") for xval in x_d];print()

 	S	N	J	B	R	Y	T	M	G	L	
S	23	32	23	34	7	7	4	12	0	0	
N	31	97	31	23	18	10	14	6	1	0	
J	28	48	20	14	14	2	7	0	0	0	
B	19	41	14	44	2	8	4	9	0	0	
R	4	21	20	15	23	0	1	7	0	0	
Y	4	32	8	2	0	12	8	5	0	0	
T	7	10	5	8	0	1	13	7	0	0	
M	3	6	0	7	1	5	1	9	1	0	
G	19	22	20	26	10	4	12	6	46	38	
L	7	14	6	4	8	2	2	0	3	4	


In [ ]:
# cell 4
# run cell 1 and 2 before this cell
# matrix to find position occurance count
# i.e, how many lakhshanam have S in position 1, G in position 3 etc...
def create_matrix(values):
    matrix = [['S','N','J','B','R','Y','T','M','G','L']]
    for value in values:
        for i in range(len(value)):
            if len(matrix)<=i+1:matrix.append([0,0,0,0,0,0,0,0,0,0])
            matrix[i+1][matrix[0].index(value[i].upper())] += 1
    return matrix

vritham_matrix = create_matrix(keydictionary.keys())
for row in vritham_matrix:
    print(row)

print("\nsum","\n"+"-"*15) # just for decoration

sum = [0,0,0,0,0,0,0,0,0,0]
for row in range(1,len(vritham_matrix)):
    for i in range(len(vritham_matrix[row])):
        sum[i] = sum[i]+vritham_matrix[row][i]
for i in range(len(sum)):
    print(vritham_matrix[0][i],"=",sum[i])

['S', 'N', 'J', 'B', 'R', 'Y', 'T', 'M', 'G', 'L']
[32, 93, 16, 37, 18, 8, 17, 34, 3, 2]
[29, 59, 44, 40, 21, 16, 20, 14, 6, 1]
[35, 72, 34, 37, 17, 8, 16, 9, 10, 5]
[33, 39, 28, 25, 25, 24, 9, 6, 32, 11]
[22, 31, 12, 21, 12, 13, 3, 3, 49, 13]
[14, 21, 10, 10, 9, 3, 2, 0, 39, 11]
[4, 7, 3, 6, 6, 5, 1, 1, 39, 2]
[4, 2, 2, 2, 1, 2, 0, 0, 15, 4]
[1, 0, 0, 0, 0, 0, 0, 0, 9, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 4, 0]

sum 
---------------
S = 174
N = 324
J = 149
B = 178
R = 109
Y = 79
T = 68
M = 67
G = 206
L = 52


In [ ]:
# cell 5
# run cell 1 and 2 before this cell
# test area
# test command: 'LLLLGLLLLLLGG'
# test command: 'MG'
command = input('>')
while not command=='':
    if check_maathra(command):
        if check_GL(command):
            maathra = get_maathra(command);print(maathra)
            testdata = keydictionary.get(maathra,'No entry Found')
        else:
            testdata = keydictionary.get(command.upper(),'No entry Found')
        print(testdata)
    else:
        print('Wrong Input Found')
    command = input('>')

>MG
['4', 'കന്യാ ', 'മഗം', '', '8', 8]
>


In [ ]:
# cell 6
def laghuguru():
    syllables=self.syllables()
    output = ['L' for syllable in syllables]
    chillu = (3450, 3451, 3452, 3453, 3454)
    g_characters = (3334, 3336, 3338, 3343, 3347, 3348, 3390, 3392, 3394, 3399, 3403, 3404, 3415, 3330, 3331)
    for index, syllable in enumerate(syllables):
        if ord(syllable[-1]) in chillu:output[index] = '-'
        elif ord(syllable[-1]) in g_characters:output[index] = 'G'
        elif len(syllable)>2 and index>0:
            if output[index-1]=='-' and index-2>=0:output[index-2]='G'
            elif output[index-1]=='L':output[index-1]='G'
            else:pass
    return ''.join(output)

def laghuguru(self):
        syllable=self.syllables()
        size=len(syllable)
        mathra=[None]*size
        for oneChar in syllable:
            if not oneChar  in ['ആ', 'ഈ', 'ഊ', 'ഏ', 'ഓ', 'ഔ']:
                if not oneChar[-1][-1] in ['ാ', 'ീ', 'ൂ', 'േ', 'ൈ', 'ോ', 'ൌ', 'ൗ', 'ം', 'ഃ']:
                    syindex=syllable.index(oneChar)
                    if (syllable.count(oneChar)>0):
                        for index, value in enumerate(syllable):
                            if (value ==oneChar):
                                mathra[index] = 'L'
        
            if (len(oneChar)>= 1):
                if oneChar[-1][-1] in ['്']:
                    if (syllable.count(oneChar)==1):
                        syindex=syllable.index(oneChar)
                        mathra[syindex]=' '
                    else:
                        for index, value in enumerate(syllable):
                            if (value ==oneChar):
                                mathra[index]=' '
                if oneChar  in ['ആ', 'ഈ', 'ഊ', 'ഏ', 'ഓ', 'ഔ']:
                    if (syllable.count(oneChar)==1):
                        syindex=syllable.index(oneChar)
                        mathra[syindex]='G'
                    else:
                        if (syllable.count(oneChar)>0):
                            for index, value in enumerate(syllable):
                                if (value ==oneChar):mathra[index] = 'G'
                if oneChar[-1][-1] in ['ാ', 'ീ', 'ൂ', 'േ', 'ൈ', 'ോ', 'ൌ', 'ൗ', 'ം', 'ഃ']:
                    if (syllable.count(oneChar)==1):
                        syindex=syllable.index(oneChar)
                        mathra[syindex]='G'
                    else:
                        if (syllable.count(oneChar)>0):
                            for index, value in enumerate(syllable):
                                if (value ==oneChar):mathra[index] = 'G'
            for oneChar in syllable:
                if (len(oneChar)> 2):
                    if oneChar[1][0] in ['്']:
                        if (syllable.count(oneChar)==1):
                            syindex=syllable.index(oneChar)
                            mathra[syindex-1]='G'
                        else:
                            for index, value in enumerate(syllable):
                                if (value ==oneChar):
                                    mathra[index-1]='G'
                                          
                if (len(oneChar)>= 2):
                    if oneChar[-1][-1] in ['ൈ']:
                        if (syllable.count(oneChar)==1):
                            syindex=syllable.index(oneChar)
                            mathra[syindex-1]='G'
                        else:
                            for index, value in enumerate(syllable):
                                if (value ==oneChar):
                                    mathra[index-1]='G'
  
       
            for oneChar in syllable:
                if oneChar in ['ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ']:
                    if (syllable.count(oneChar)==1):
                        syindex=syllable.index(oneChar)
                        mathra[syindex]=' '
                    else:
                        if (syllable.count(oneChar)>0):
                            for index, value in enumerate(syllable):
                                if (value ==oneChar):mathra[index] = ' '

        #print(mathra,end='')
        return mathra
"""    This code still work. Storing just for reference
    def laghuguru(self):
        mathra = [None for i in range(len(self))]
        Hswara_array = [chr(swara) for swara in [3334, 3336, 3338, 3343, 3347, 3348]]
        Hsign_array = [chr(sign) for sign in [3390, 3392, 3394, 3399, 3400, 3403, 3404, 3415, 3330, 3331]]
        chilu_array = [chr(chilu) for chilu in [3450, 3451, 3452, 3453, 3454]]
        this_syllables = self.syllables()
        for oneChar in this_syllables:
            if not oneChar in Hswara_array:
                if not oneChar[-1][-1] in Hsign_array:
                    if (this_syllables.count(oneChar)>0):
                        for index, value in enumerate(this_syllables):
                            if (value == oneChar):mathra[index] = 'L'
            else:
                for index, value in enumerate(this_syllables):
                    if value == oneChar:mathra[index] = 'G'

            if oneChar[-1][-1] in Hsign_array:
                for index, value in enumerate(this_syllables):
                    if value == oneChar:mathra[index] = 'G'

            if (len(oneChar)>2):
                if oneChar[1][0] in [chr(c) for c in [3405]]:
                    for index, value in enumerate(this_syllables):
                        if value == oneChar:
                            mathra[index-1] = 'G'
                            mathra[index] = 'L'

            if (len(oneChar)>=2):
                if oneChar[-1][-1] in [chr(c) for c in [3400]]:
                    for index, value in enumerate(this_syllables):
                        if value == oneChar:mathra[index-1] = 'G'

            if len(oneChar)>=1:
                if oneChar[-1][-1] in [chr(c) for c in [3405]]:
                    for index, value in enumerate(this_syllables):
                        if value == oneChar:mathra[index] = 'L'

            if oneChar in chilu_array:
                for index, value in enumerate(this_syllables):
                    if value == oneChar:mathra[index] = ' '
        return mathra
"""

A class for dealing syllables and laghuguru codes together

In [3]:
# cell 7
class ml_word:
    def __init__(self, word):
        self.word = word

    def syllables(self):
        sign = [3330, 3331, 3390, 3391, 3392, 3393, 3394, 3395, 3396,
                3398, 3399, 3400, 3402, 3403, 3404, 3405, 3415]
        output = [];connected = False;word_len = len(self.word)
        for index in range(word_len):
            if ord(self.word[index])<3330 or ord(self.word[index])>3455:connected = False;continue
            if not connected:output.append(self.word[index])
            else:output[-1] += self.word[index]
            if index+1 >= word_len:continue
            elif ord(self.word[index+1]) in sign:connected = True
            elif ord(self.word[index]) in [3405]:
                nonsigncharacters = ""
                for character in output[-1]:
                    if ord(character) not in sign:nonsigncharacters = nonsigncharacters + character
                if output[-1].count(chr(3405))<2:connected = True
                elif (ord(self.word[index+1]) in [i for i in range(3375,3386)]):
                    if len(nonsigncharacters)<3:connected = True
                    else:connected = False
                else:
                    connected = False
                    for character in nonsigncharacters:
                        if (ord(character) in [i for i in range(3375,3386)]):
                            connected = True
                            break
            elif ord(self.word[index]) in [3451]:connected = True if ord(self.word[index+1])==3377 else False
            else:connected = False
        return output

    def laghuguru(self):
        def nonsignchars(syllable):
            signs = (3330, 3331, 3390, 3391, 3392, 3393, 3394, 3395, 3396,3398, 3399, 3400, 3402, 3403, 3404, 3405, 3415)
            output = [s for s in syllable if ord(s) not in signs]
            return ''.join(output)
        syllables = self.syllables()
        output = ['L' for syllable in syllables]
        chillu = (3450, 3451, 3452, 3453, 3454)
        g_characters = (3334, 3336, 3338, 3343, 3347, 3348, 3390, 3392, 
                        3394, 3399, 3400, 3403, 3404, 3415, 3330, 3331)
        for index, syllable in enumerate(syllables):
            if ord(syllable[-1]) in chillu:output[index] = '-'
            elif ord(syllable[-1]) in g_characters:output[index] = 'G'
            if len(nonsignchars(syllable))>=2 and index>0:
                if output[index-1]=='-' and index-2>=0:output[index-2]='G'
                elif output[index-1]=='L':output[index-1]='G'
                else:pass
            if ord(syllable[-1])==3405:output[index]='-' # convert character end in chandrakala into -                                                                                 
        if len(output)>1 and output[-1]=='-':output[-2]='G'
        return output

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word

    def __iter__(self):
        for char in self.syllables():
            yield char
    def __len__(self):
        return len(self.syllables())

In [ ]:
# cell 8
# run cell 1, 2 and 7 before this cell
#test
word = ml_word(str(input("Input: ")))
syllables = word.syllables()
mathra = word.laghuguru()
option = int(input(" Your option : "))
if option == 1:pass
elif option == 2:mathra[-1] = 'G'
else:print("Default 1")
table_dictionary = {'Syllable':(syllables), 'Mathra':(mathra)}
df=pd.DataFrame(table_dictionary)
df=df.T
display(df.style)

Input: നില്ക്കേണം
 Your option : 2


,0,1,2
Syllable,നി,ല്ക്കേ,ണം
Mathra,G,G,G


Following code load example kavitha/film songs then show data from it

In [ ]:
# cell 9
# run cell 1, 2 and 7 before this cell

def alignwithspace(text,syllable): # left align a text
    length = len(syllable)
    sign = [3330, 3331, 3390, 3391, 3392, 3393, 3394, 3395, 3396,
            3398, 3399, 3400, 3402, 3403, 3404, 3405, 3415] # if sign lack space
    output = ""
    sign_count = len(syllable)
    for i in range(0,length):
        if ord(syllable[i]) in sign:sign_count-=1
        if i<len(text):output+=text[i]
        else:output+=" "
    return output[0:sign_count]+"  " # + 2 space

#formattext - This code align text for easy reading 
def formattext(syllables,mathra):
    output = []
    temp_sy = [];temp_lg = []
    limit = 20
    for index, syllable in enumerate(syllables):
        temp_sy.append(syllable)
        #temp_lg.append(mathra[index].replace(" ",'-'))
        temp_lg.append(alignwithspace(mathra[index].replace(" ",'-'),syllable))
        if ((len(temp_sy)==limit) or(index+1==len(syllables))):
            output.append(temp_sy);output.append(temp_lg)
            temp_sy = [];temp_lg = []
    return output

def changelastLAGHUtoGURU(laghugurutext):
    for i in range(0,len(laghugurutext)):
        if laghugurutext[-1-i].upper() == 'L':
            if i==0:return laghugurutext[0:-1]+'G'
            else:return laghugurutext[0:-1-i]+'G'+laghugurutext[-i:]
    return laghugurutext

def sort_second_column(arr):
    output = sorted(arr, key=lambda x: x[1][4])
    return output

def sort_output(arr):
    first_column_sorted = sorted(arr, key=lambda x: x[0][4])
    second_column_sorted = []
    temp_second = []
    prev_val = -1
    for index, item in enumerate(first_column_sorted):
        if index+1>=len(first_column_sorted):
            if item[0][4]==prev_val:temp_second.append(item)
            else:
                if len(temp_second)>0:second_column_sorted.extend(sort_second_column(temp_second))
                temp_second = [];temp_second.append(item)
            second_column_sorted.extend(sort_second_column(temp_second))
        else:
            if item[0][4]==prev_val:temp_second.append(item)
            else:
                prev_val = item[0][4]
                if len(temp_second)>0:second_column_sorted.extend(sort_second_column(temp_second))
                temp_second = [];temp_second.append(item)
    new_output = []
    for row in second_column_sorted:
        temp_row = []
        for column in row:
            temp_column = []
            for item in column:
                temp_column.append(str(item))
            temp_row.append("".join(temp_column))
        new_output.append(temp_row)
    return new_output
        
lakshanam_collections = [] 

examplefilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/example2.csv'
examplecsvresponse = requests.get(examplefilename)
examplecsvfile = codecs.iterdecode(examplecsvresponse.iter_lines(), 'UTF-8')
examplecsvreader = csv.reader(examplecsvfile)
with open('samples.csv', 'w', encoding='utf-8') as outputfile:
    examplecsvwriter = csv.writer(outputfile)
    word = ml_word("")
    output_data = []
    for row in examplecsvreader:
        row_data = []
        for column_index, column in enumerate(row):
            word.word = column
            syllables = word.syllables()
            laghuguru = "".join(word.laghuguru())
            if column_index == 1:
                laghuguru = changelastLAGHUtoGURU(laghuguru) # last one changed to guru --  
            laghuguru_nospace = laghuguru.replace(" ","")   
            mathra = calculatemathra(laghuguru_nospace)
            syllable_length = len(syllables)
            lakshanam = get_maathra(laghuguru_nospace)  
            vritham = keydictionary.get(lakshanam.upper(),[])
            for index_i, item_x in enumerate(vritham):
                vritham[index_i] = str(item_x)
            formattedtext = formattext(syllables,laghuguru)
            output_text = ""
            for text in formattedtext:
                output_text += " ".join(text)+";"
            #", ".join(syllables)+";"+str(laghuguru)
            if len(lakshanam)>0:lakshanam_collections.append(lakshanam)
            if len(vritham)>0:
                row_data.append([column,";",output_text,"M",mathra," S",str(syllable_length)," ",str(lakshanam),";"," ".join(vritham)])
            else:
                row_data.append([column,";",output_text,"M",mathra," S",str(syllable_length)," ",str(lakshanam)])
        output_data.append(row_data)
    final_output = sort_output(output_data)
    examplecsvwriter.writerows(final_output)


In [ ]:
# cell 10
# run cell 1 and 2 before this cell
entries = ['S','N','J','B','R','Y','T','M','G','L']
vritham_matrix = create_matrix()
kmers = create_kmers(lakshanam_collections)
print(len(kmers))
for kmer in kmers:
    x = entries.index(kmer[0].upper())
    y = entries.index(kmer[1].upper())
    vritham_matrix[y][x]+=1
y_arr = [' '];y_arr.extend(entries)
y = [print(str(yval),end="\t")for yval in y_arr];print()
for row_ind in range(len(vritham_matrix)):
    x_d = [entries[row_ind]];x_d.extend(vritham_matrix[row_ind])
    x = [print(str(xval),end="\t") for xval in x_d];print()

2924
 	S	N	J	B	R	Y	T	M	G	L	
S	51	27	18	37	51	17	22	42	0	0	
N	32	59	18	37	24	18	17	14	0	0	
J	32	34	25	33	44	24	25	19	0	0	
B	25	29	28	83	46	16	40	26	0	0	
R	29	27	33	52	78	14	58	31	0	0	
Y	31	35	24	21	30	38	32	25	0	0	
T	27	25	28	48	40	7	53	23	0	0	
M	35	32	33	44	53	21	45	57	0	0	
G	44	27	32	35	64	57	33	92	164	44	
L	23	11	8	12	28	16	9	17	43	23	


Updated Guru Laghu 14-10-2021

In [ ]:
# cell 11
import pandas as pd
def switch():
  vakku = input("Enter the word/sloka : ")
  print("Enter 1 for checking Mathra in a word \n Enter 2 for checking mathras of  Eradi or sloka\n")
  option = int(input(" Your option : "))
  def word():
    syllable,mathra=laghuguru(vakku)
    table_dictionary = {'Syllable':(syllable), 'Mathra':(mathra)}
    df=pd.DataFrame(table_dictionary)
    df=df.T
    display(df.style)
   
  def sloka():
    syllable,mathra=laghuguru(vakku)
    x=syllable[-1]
    if x in ['ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ']:
      mathra[-1]=' '
    else:
      mathra[-1]='G'
    table_dictionary = {'Syllable':(syllable), 'Mathra':(mathra)}
    df=pd.DataFrame(table_dictionary)
    df=df.T
    display(df.style)
  def syllables(word):
    sign = [3330, 3331, 3390, 3391, 3392, 3393, 3394, 3395, 3396,3398, 3399, 3400, 3402, 3403, 3404, 3405, 3415]
    output = [];connected = False;word_len = len(word)
    for index in range(word_len):
      if ord(word[index])<3330 or ord(word[index])>3455:connected = False;continue
      if not connected:output.append(word[index])
      else:output[-1] += word[index]
      if index+1 >= word_len:continue
      elif ord(word[index+1]) in sign:connected = True
      elif ord(word[index]) in [3405]:
        nonsigncharacters = ""
        for character in output[-1]:
          if ord(character) not in sign:nonsigncharacters = nonsigncharacters + character
        if output[-1].count(chr(3405))<2:connected = True
        elif (ord(word[index+1]) in [i for i in range(3375,3386)]):
          if len(nonsigncharacters)<3:connected = True
          else:connected = False
        else:
          connected = False
          for character in nonsigncharacters:
            if (ord(character) in [i for i in range(3375,3386)]):
              connected = True
              break
      elif ord(word[index]) in [3451]:connected = True if ord(word[index+1])==3377 else False
      else:connected = False
    return output
  def laghuguru(word):
    syllable=syllables(word)
    #print(syllable)
    size=len(syllable)
    mathra=[None]*size
    for oneChar in syllable:
      if not oneChar  in ['ആ', 'ഈ', 'ഊ', 'ഏ', 'ഓ', 'ഔ']:
            if not oneChar[-1][-1] in ['ാ', 'ീ', 'ൂ', 'േ', 'ൈ', 'ോ', 'ൌ', 'ൗ', 'ം', 'ഃ']:
                syindex=syllable.index(oneChar)
                if(syllable.count(oneChar)>0):
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):
                            mathra[index] = 'L'
        
      if (len(oneChar)>= 1):
        if oneChar[-1][-1] in ['്']:
          if (syllable.count(oneChar)==1):
            syindex=syllable.index(oneChar)
            mathra[syindex]=' '
          else:
            for index, value in enumerate(syllable):
              if (value ==oneChar):
                mathra[index]=' '
        if oneChar  in ['ആ', 'ഈ', 'ഊ', 'ഏ', 'ഓ', 'ഔ']:
          if (syllable.count(oneChar)==1):
            syindex=syllable.index(oneChar)
            mathra[syindex]='G'
          else:
            if(syllable.count(oneChar)>0):
              for index, value in enumerate(syllable):
                if (value ==oneChar):mathra[index] = 'G'
        if oneChar[-1][-1] in ['ാ', 'ീ', 'ൂ', 'േ', 'ൈ', 'ോ', 'ൌ', 'ൗ', 'ം', 'ഃ']:
            if (syllable.count(oneChar)==1):
                syindex=syllable.index(oneChar)
                mathra[syindex]='G'
            else:
                if(syllable.count(oneChar)>0):
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):mathra[index] = 'G'
    for oneChar in syllable:
        if (len(oneChar)> 2):
            if oneChar[1][0] in ['്']:
                if (syllable.count(oneChar)==1):
                    syindex=syllable.index(oneChar)
                    mathra[syindex-1]='G'
                else:
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):
                          mathra[index-1]='G'
                                          
        if (len(oneChar)>= 2):
            if oneChar[-1][-1] in ['ൈ']:
                if (syllable.count(oneChar)==1):
                    syindex=syllable.index(oneChar)
                    mathra[syindex-1]='G'
                else:
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):
                            mathra[index-1]='G'
  
    for oneChar in syllable:
        if (len(oneChar)> 2):
            if oneChar[1][0] in ['്']:
                if (syllable.count(oneChar)==1):
                    syindex=syllable.index(oneChar)
                    if (syllable[syindex-1]) in ['ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ']:
                      mathra[syindex-2]='G'

                else:
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):
                          if (syllable[syindex-1]) in ['ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ']:
                            mathra[index-2]='G'   
    for oneChar in syllable:
        if oneChar in ['ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ']:
          chillu=oneChar
          sp = input("whether chill %s needs to be strongly pronunced (y/n) : " % chillu)
          if sp=='y':
            if (syllable.count(oneChar)==1):
                syindex=syllable.index(oneChar)
                mathra[syindex-1]='G'
                mathra[syindex]=' '
            else:
                if(syllable.count(oneChar)>0):
                    for index, value in enumerate(syllable):
                        if (value ==oneChar):
                          mathra[index] = ' '
                          mathra[index-1] = 'G'

          if sp=='n':
            if (syllable.count(oneChar)==1):
              syindex=syllable.index(oneChar)
              mathra[syindex]=' '
            else:
              if(syllable.count(oneChar)>0):
                for index, value in enumerate(syllable):
                  if (value ==oneChar):mathra[index] = ' '
    return(syllable,mathra) 
  def default():
    print("Incorrect option")
 
# Dictionary Mapping
  dict = {1 : word,2 : sloka}
  dict.get(option,default)() 
switch()  

Enter the word/sloka : ഭൂമിയ്ക്കു നല്കിടും ഭാഗ്യവും നന്മയും 
Enter 1 for checking Mathra in a word 
 Enter 2 for checking mathras of  Eradi or sloka

 Your option : 2


,0,1,2,3,4,5,6,7,8,9,10,11
Syllable,ഭൂ,മി,യ്ക്കു,ന,ല്കി,ടും,ഭാ,ഗ്യ,വും,ന,ന്മ,യും
Mathra,G,G,L,G,L,G,G,L,G,G,L,G


kavithaset (malayalam kavitha database) processing with docx output

In [ ]:
# cell 12
# run cells 2 and 7 before this cell
kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/kavithasets.csv' # malayalam kavitha dataset (2695 examples)
csvresponse = requests.get(kavithafilename)
csvfile = codecs.iterdecode(csvresponse.iter_lines(), 'UTF-8')
csvreader = csv.reader(csvfile)

kavithadata = []
index = 0

word = ml_word("") # class for handling malayalam words
for row in tqdm(csvreader):
    index = index + 1
    input_text, poet = row[0], row[2]
    word.word = input_text
    syllables_raw = word.syllables()       # extract syllables
    lg_raw = word.laghuguru()              # extract laghuguru
    syllables = ','.join(syllables_raw)    # format syllables
    laghuguru = ' '.join(lg_raw) 
    lg_nosign = laghuguru.replace(' ','').replace('-','') # remove signs from laghuguru to find lakshanam and calculate maathra
    lakshanam = lakshanam = get_maathra(lg_nosign)
    vritham = keydictionary.get(lakshanam.upper(),['','',''])
    if vritham == ['','','']:
        curr_score = 0
        input_a = converttoGL(lakshanam)
        for key in keydictionary.keys():
            input_b = converttoGL(key)
            if difflib.SequenceMatcher(None, input_a, input_b).ratio()>curr_score:
                curr_score = difflib.SequenceMatcher(None, input_a, input_b).ratio()
                vritham = [curr_score, keydictionary[key][1], key,'prediction']
    syllable_count = len(syllables_raw)-''.join(lg_raw).count('-') # avoid - in maathra while counting syllables
    try:
        if vritham[3]=='prediction':
            kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]+' (Match='+str(round(vritham[0],2))+')']])
        else:
            kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]]])
    except:
        kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]]])
        print(vritham)


2631it [01:08, 46.85it/s]

['', '', '']


2696it [01:10, 38.31it/s]


In [8]:
# cell 13
# run cells 2 and 7 before this cell
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/nathonnatha.csv' # nathonnatha dataset (164 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/nathonnatha2.csv' # nathonnatha2 dataset (154 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/keka.csv' # keka dataset (117 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/kakali.csv' # kakali dataset (331 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/kummi.csv' # kummi dataset (14 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/kurathi.csv' # kurathi dataset (60 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/kakali2.csv' # kakali dataset 2 (307 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/maveli-kummi.csv' # maveli-kummi dataset (175 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/test-data/keka- nathonatha-sample.csv' # keka - nathonatha sample dataset (8 examples)

#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/data/maveli-kummi-v7.csv' # maveli-kummi V7 dataset (167 examples)
#kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/data/omanakuttan-v2.csv' # omanakuttan V2 dataset (40 examples)
kavithafilename = 'https://raw.githubusercontent.com/dcbfoss/vritham/test/data/kakali-v3.csv' # kakali V3 dataset (311 examples)
csvresponse = requests.get(kavithafilename)
csvfile = codecs.iterdecode(csvresponse.iter_lines(), 'UTF-8')
csvreader = csv.reader(csvfile)

kavithadata = []
index = 0

word = ml_word("") # class for handling malayalam words
for row in tqdm(csvreader):
    index = index + 1
    input_text, poet, line_number = row[0], row[2], row[3]
    word.word = input_text
    syllables_raw = word.syllables()       # extract syllables
    syllables = ','.join(syllables_raw)    # format syllables
    lg_raw = word.laghuguru()              # extract laghuguru
    try:
        if int(line_number)%2==0 and lg_raw[-1]=='L':lg_raw[-1]='G'
    except:pass
    laghuguru = '  '.join(lg_raw) 
    lg_nosign = laghuguru.replace(' ','').replace('-','') # remove signs from laghuguru to find lakshanam and calculate maathra
    lakshanam = lakshanam = get_maathra(lg_nosign)
    vritham = keydictionary.get(lakshanam.upper(),['','',''])
    if vritham == ['','','']:
        curr_score = 0
        input_a = converttoGL(lakshanam)
        for key in keydictionary.keys():
            input_b = converttoGL(key)
            if difflib.SequenceMatcher(None, input_a, input_b).ratio()>curr_score:
                curr_score = difflib.SequenceMatcher(None, input_a, input_b).ratio()
                vritham = [curr_score, keydictionary[key][1], key,'prediction']
    syllable_count = len(syllables_raw)-''.join(lg_raw).count('-') # avoid - in maathra while counting syllables
    try:
        if vritham[3]=='prediction':
            kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]+' (Match='+str(round(vritham[0],2))+')']])
        else:
            kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]]])
    except:
        kavithadata.append([[str(index)],[laghuguru,syllables],[str(syllable_count)],[str(calculatemathra(lg_nosign))],[lakshanam,vritham[1]]])
        print(vritham)

312it [00:05, 57.04it/s]


In [5]:
# cell 14
# working with docx
!pip install python-docx
from docx import Document
from docx.enum.section import WD_ORIENT

     |████████████████████████████████| 5.6 MB 4.5 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=71b4263281512770b6eaf0aad6b448b1210dd455aa13afb2c24052275c06302a
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [9]:
# cell 15
# run cells 12 or 13 and 14 before this cell

rowcount = len(kavithadata) # calculated from data in cell before
headers = ('No.','','S','M','Vritham')

# create a document object
kavyaoutput = Document()

# change orientation to landscape
section = kavyaoutput.sections[-1]
section.orientation = WD_ORIENT.LANDSCAPE
section.page_height, section.page_width = section.page_width, section.page_height

# adding table and header data
kavyatable = kavyaoutput.add_table(rows=rowcount+1, cols=len(headers))
kavyaheader = kavyatable.rows[0].cells
for index, header in enumerate(headers):
    kavyaheader[index].paragraphs[0].add_run(header).bold=True

# writing data
for i in tqdm(range(rowcount)):
    thisrow = kavyatable.rows[i+1].cells
    for j in range(len(headers)):
        if len(kavithadata[i][j])>1:thisrow[j].text = "\n".join(kavithadata[i][j])
        else:thisrow[j].text = str(kavithadata[i][j][0])

# saving document

#kavyaoutput.save('kavyaoutput.docx')
#kavyaoutput.save('nathonnathaoutput.docx')
#kavyaoutput.save('kekaoutput.docx')
kavyaoutput.save('kakalioutput.docx')
#kavyaoutput.save('kummioutput.docx')
#kavyaoutput.save('kurathioutput.docx')
#kavyaoutput.save('kakali2output.docx')
#kavyaoutput.save('maveli-kummi.docx')
#kavyaoutput.save('omanakuttan.docx')
#kavyaoutput.save('keka-nathonatha-sample.docx')

100%|██████████| 312/312 [00:12<00:00, 24.02it/s]


In [ ]:
# cell 16
# run cell 1 and 2 (with sanskrit db) before this cell
s1=["GLL", "GGL", "GGG", "GLG", "LGG"]
s2=["GG", "GL", "LG"]
s3=["GG", "GL", "LG"]
s4=["GLL", "GGL", "GGG", "GLG", "LGG"] 
s5=["GG", "GL", "LG"]
s6=["GG", "GL", "LG"]
kekaGL = [];output_match=[] # keka vritham GL strings (2025 combinations)
for a in s1:
    for b in s2:
        for c in s3:
            for d in s4:
                for e in s5:
                    for f in s6:
                        kekaGL.append(a+b+c+d+e+f)
# Rule 1: find any match in sanskrit only database
# Rule 2: if match was not found, report similar if similarity>0.9
print("reading database")
for glentry in tqdm(kekaGL):
    vritham = keydictionary.get(lakshanam.upper(),['','','',''])
    if vritham == ['','','','']:
        curr_score = 0
        for key in keydictionary.keys():
            glkey = converttoGL(key)
            if difflib.SequenceMatcher(None, glentry, glkey).ratio()>curr_score:
                curr_score = difflib.SequenceMatcher(None, glentry, glkey).ratio()
                if curr_score>=0.9:vritham = [curr_score, keydictionary[key][1], key,'prediction']
    if vritham == ['','','','']:output_match.append("--")
    elif vritham[3]== 'prediction':output_match.append(str(round(vritham[0],2))+";"+vritham[1])
    else:output_match.append(vritham[1])
print("\nwriting output")
with open("kekaGLstringcomp.csv",'w') as outfile:
    for i, x in enumerate(kekaGL):
        outfile.write(x+","+output_match[i]+"\n")
    findings = {}
    for j in output_match:
        if ';' in j:
            found = j.split(sep=';')[1]
            findings[found] = findings.get(found,0) + 1
    for k in findings.keys():
        outfile.write(k+","+str(findings[k])+"\n")
    print("writing completed")

reading database


100%|██████████| 2025/2025 [00:42<00:00, 47.95it/s]


writing output
writing completed


In [ ]:
# cell 17
# run cell 2 before this cell

# find my relative

inputlakshanam = input("Enter a lakshanam:")
print(converttoGL(inputlakshanam)) # test

def alignmatrix(a,b):
    matrix = []; gap_penality = -1
    mismatch = -1; match = 1
    for i in range(len(b)+1):
        matrix.append([])
        for j in range(len(a)+1):
            if i==0:matrix[i].append(0-j)
            elif j==0:matrix[i].append(0-i)
            else:matrix[i].append(0)
    for i in range(len(b)):
        for j in range(len(a)):
            score = match if b[i]==a[j] else mismatch
            score = score + matrix[i][j]
            top_gap = matrix[i][j+1] + gap_penality
            side_gap = matrix[i+1][j] + gap_penality
            matrix[i+1][j+1] = max(score, top_gap, side_gap)
    return matrix

def tracebackmatrix(matrix, a, b):
    str1 = '' # store b
    str2 = '' # store a
    x = len(b)
    y = len(a)
    mismatch = -1; match = 1;gap_penality = -1
    while x>0 or y>0:
        score = match if b[x-1]==a[y-1] else mismatch
        if (x > 0 and y > 0 and matrix[x][y]==score+matrix[x-1][y-1]):
            str1 = b[x-1] + str1
            str2 = a[y-1] + str2
            x -= 1;y -= 1
        elif (x > 0 and matrix[x][y]==gap_penality+matrix[x-1][y]):
            str1 = b[x-1] + str1
            str2 = '_' + str2
            x -= 1
        else:
            str1 = '_' + str1
            str2 = a[y-1] + str2
            y -= 1
    return (str2, str1)


Enter a lakshanam:RSTR
GLGLLGGGLGLG


similarity check 

In [ ]:
import difflib

print(difflib.SequenceMatcher(None, "GLGLLGGGLGLG", "GLGLLGGGLGLG").ratio())

1.0


In [ ]:
print(difflib.SequenceMatcher(None, "GLGLLGGGLGLG", "GLGLLGGGLGLL").ratio())

0.9166666666666666


In [ ]:
from difflib import SequenceMatcher
str1 = 'abcd'
str2 = 'abcde'
seq = SequenceMatcher(a=str1, b=str2)
print(seq.ratio())

0.8888888888888888


EXp 2:

In [ ]:
import numpy as np

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    print (matrix)
    return (matrix[size_x - 1, size_y - 1])

In [ ]:
levenshtein("GLG", "GLGLLLLLLLL")

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [ 1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 2.  1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 3.  2.  1.  0.  1.  2.  3.  4.  5.  6.  7.  8.]]


8.0

In [ ]:
levenshtein("GLGLLGGGLGLG", "GLG")

[[ 0.  1.  2.  3.]
 [ 1.  0.  1.  2.]
 [ 2.  1.  0.  1.]
 [ 3.  2.  1.  0.]
 [ 4.  3.  2.  1.]
 [ 5.  4.  3.  2.]
 [ 6.  5.  4.  3.]
 [ 7.  6.  5.  4.]
 [ 8.  7.  6.  5.]
 [ 9.  8.  7.  6.]
 [10.  9.  8.  7.]
 [11. 10.  9.  8.]
 [12. 11. 10.  9.]]


9.0

In [ ]:
pip install textdistance

In [ ]:
import textdistance

textdistance.hamming('test', 'text')
# 1

textdistance.hamming.distance('test', 'text')
# 1

textdistance.hamming.similarity('test', 'text')
# 3

textdistance.hamming.normalized_distance('test', 'text')
# 0.25

textdistance.hamming.normalized_similarity('test', 'text')
# 0.75

textdistance.Hamming(qval=2).distance('test', 'text')
# 2

2

In [ ]:
import textdistance
hamming = textdistance.Hamming(external=False)
hamming('text', 'testit')

3

In [ ]:
import textdistance
hamming = textdistance.	NeedlemanWunsch(external=False)
hamming('text', 'testit')

1.0

In [ ]:
import textdistance
hamming = textdistance.Hamming(external=False)
hamming('GLGLLGGGLGLG', 'GLGGLGLL')

7